In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
import os

src_dir = '/content/drive/MyDrive/Colab Notebooks/facial'
dst_dir = '/content/'

for item in os.listdir(src_dir):
    s = os.path.join(src_dir, item)
    d = os.path.join(dst_dir, item)
    if os.path.isdir(s):
        shutil.copytree(s, d, dirs_exist_ok=True)
    else:
        shutil.copy2(s, d)


In [ ]:
!nvidia-smi

Sun Jun  1 10:11:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             48W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!git clone https://github.com/minhnhatvt/label-distribution-learning-fer-tf.git
%cd label-distribution-learning-fer-tf

Cloning into 'label-distribution-learning-fer-tf'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 65 (delta 26), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 27.89 KiB | 125.00 KiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/label-distribution-learning-fer-tf


In [ ]:
!chmod +x install.sh
!./install.sh

./install.sh: line 1: conda: command not found
  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-2oqoz63a
  Running command git clone --filter=blob:none --quiet https://github.com/qubvel/classification_models.git /tmp/pip-req-build-2oqoz63a
  Resolved https://github.com/qubvel/classification_models.git to commit a0f006e05485a34ccf871c421279864b0ccd220b
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.1 MB/s eta 0:00:00
  Created wheel for image_classifiers: filename=image_classifiers-1.0.0-py3-none-any.whl size=20031 sha256=63e12e2b5e8be68801a0db83257d96ce8090202235544860c7c2b0578babc183
  Stored in directory: /tmp/pip-ephem-wheel-cache-ksrd1ieu/wheels/14/43/ed/2d68b197c5260e7fe6b7d0fab20b997b6d365f6f5732316952
Successfully built image_classifiers


In [ ]:
import os

os.makedirs("data/rafdb", exist_ok=True)
!unzip -q /content/aligned.zip -d data/rafdb/


In [ ]:
os.makedirs('pretrained', exist_ok=True)
shutil.move('/content/resnet101.h5', 'pretrained/resnet101.h5')
shutil.move('/content/resnet18.h5', 'pretrained/resnet18.h5')
shutil.move('/content/resnet50.h5', 'pretrained/resnet50.h5')

'pretrained/resnet50.h5'

In [ ]:
shutil.move('/content/raf_test.csv', 'data/rafdb/raf_test.csv')
shutil.move('/content/raf_train.csv', 'data/rafdb/raf_train.csv')

'data/rafdb/raf_train.csv'

In [ ]:
!sed -i 's/lr=/learning_rate=/' /content/label-distribution-learning-fer-tf/src/utils.py

In [ ]:
!sed -i "s/tf.keras.optimizers.SGD(config.lamb_lr, config.lamb_beta)/tf.keras.optimizers.SGD(learning_rate=float(config.lamb_lr), momentum=float(config.lamb_beta))/" /content/label-distribution-learning-fer-tf/src/train.py

In [ ]:
!sed -i 's/reset_states()/reset_state()/g' /content/label-distribution-learning-fer-tf/src/train.py

In [ ]:
!sed -i "s/float(optimizer.learning_rate(optimizer.iterations))/float((optimizer.learning_rate(optimizer.iterations) if callable(optimizer.learning_rate) else optimizer.learning_rate).numpy())/" /content/label-distribution-learning-fer-tf/src/train.py

In [ ]:
!sed -i "s|model.save_weights(f'{ckpt_dir}/epoch_' + str(curr_epoch) + '/Model')|model.save_weights(f'{ckpt_dir}/epoch_{curr_epoch}/Model.weights.h5')|" /content/label-distribution-learning-fer-tf/src/train.py


In [ ]:
!ls /content/label-distribution-learning-fer-tf/src/train.py
!grep "model.save_weights" /content/label-distribution-learning-fer-tf/src/train.py

/content/label-distribution-learning-fer-tf/src/train.py
                    model.save_weights(f"{ckpt_dir}/best_val/Model")
            model.save_weights(f'{ckpt_dir}/epoch_{curr_epoch}/Model.weights.h5')
                model.save_weights(f"{ckpt_dir}/best_val/Model")


In [ ]:
!sed -i "s|model.save_weights(f'{ckpt_dir}/epoch_{curr_epoch}/Model.weights.h5')|os.makedirs(f'{ckpt_dir}/epoch_{curr_epoch}', exist_ok=True); model.save_weights(f'{ckpt_dir}/epoch_{curr_epoch}/Model.weights.h5')|" /content/label-distribution-learning-fer-tf/src/train.py


In [ ]:
!grep "save_weights" /content/label-distribution-learning-fer-tf/src/train.py

                    model.save_weights(f"{ckpt_dir}/best_val/Model")
            os.makedirs(f'{ckpt_dir}/epoch_{curr_epoch}', exist_ok=True); model.save_weights(f'{ckpt_dir}/epoch_{curr_epoch}/Model.weights.h5')
                model.save_weights(f"{ckpt_dir}/best_val/Model")


In [ ]:
!sed -i "s/'epochs': *[0-9]\+/'epochs': 5/" /content/label-distribution-learning-fer-tf/cfg_files/config_resnet50_raf.py


In [ ]:
!grep "'epochs'" /content/label-distribution-learning-fer-tf/cfg_files/config_resnet50_raf.py


    'epochs': 5,


In [ ]:
# !sed -i "s/'batch_size': *[0-9]\+/'batch_size': 16/" /content/label-distribution-learning-fer-tf/cfg_files/config_resnet50_raf.py


In [ ]:
# !grep "'batch_size'" /content/label-distribution-learning-fer-tf/cfg_files/config_resnet50_raf.py


In [ ]:
!python src/train.py \
  --cfg=config_resnet50_raf \
  --train_data_path=data/rafdb/raf_train.csv \
  --train_image_dir=data/rafdb/aligned

2025-06-01 13:33:16.171159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748784796.191148   54664 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748784796.197162   54664 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
{'name': 'Base Config', 'backbone': 'resnet50', 'feature_dim': 512, 'pretrained': 'msceleb', 'input_size': [112, 112], 'pad_size': 4, 'batch_size': 16, 'num_parallel_calls': -1, 'optimizer': 'adam', 'lr': 0.0001, 'lr_decay': 0.1, 'lr_steps': [10, 30], 'gamma': 0.01, 'num_neighbors': 8, 'lamb_init': 0.5, 'lamb_lr': 10, 'lamb_beta': 0, 'num_classes': 7, 'class_names': ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger'],

In [ ]:
!python src/eval.py --cfg=config_resnet50_raf --trained_weights=trained_weights/trained_resnet50_raf --test_data_path=data/rafdb/raf_test.csv --test_image_dir=data/rafdb/aligned

In [ ]:

!zip -r original_facial.zip label-distribution-learning-fer-tf

!mkdir -p /content/drive/MyDrive/original_facial

!mv original_facial.zip /content/drive/MyDrive/original_facial/


In [ ]:
!ls -R label-distribution-learning-fer-tf